# Scipy Sparse Matrices

**Sparse Matrices** are very nice in some situations.  

For example, in some machine learning tasks, especially those associated
with textual analysis, the data may be mostly zeros.  

Storing all these zeros is very inefficient.  

We can create and manipulate sparse matrices as follows:

In [ ]:
import numpy as np

In [ ]:
# Create a random array with a lot of zeros
X = np.random.random((10, 5))
print(X)

In [ ]:
X[X < 0.7] = 0
print(X)

In [ ]:
from scipy import sparse

# turn X into a csr (Compressed-Sparse-Row) matrix
X_csr = sparse.csr_matrix(X)
print(X_csr)

In [ ]:
# convert the sparse matrix to a dense array
print(X_csr.toarray())

In [ ]:
# Sparse matrices support linear algebra:
y = np.random.random(X_csr.shape[1])
z1 = X_csr.dot(y)
z2 = X.dot(y)
np.allclose(z1, z2)

* The CSR representation can be very efficient for computations, but it is not as good for adding elements.  

* For that, the **LIL** (List-In-List) representation is better:

In [ ]:
# Create an empty LIL matrix and add some items
X_lil = sparse.lil_matrix((5, 5))

for i, j in np.random.randint(0, 5, (15, 2)):
    X_lil[i, j] = i + j

print(X_lil)
print(X_lil.toarray())

* Often, once an LIL matrix is created, it is useful to convert it to a CSR format 
    * **Note**: many scikit-learn algorithms require CSR or CSC format

In [ ]:
X_csr = X_lil.tocsr()
print(X_csr)

There are several other sparse formats that can be useful for various problems:

- `CSC` (compressed sparse column)
- `BSR` (block sparse row)
- `COO` (coordinate)
- `DIA` (diagonal)
- `DOK` (dictionary of keys)

## CSC - Compressed Sparse Column

**Advantages of the CSC format**

    * efficient arithmetic operations CSC + CSC, CSC * CSC, etc.
    * efficient column slicing
    * fast matrix vector products (CSR, BSR may be faster)

**Disadvantages of the CSC format**

    * slow row slicing operations (consider CSR)
    * changes to the sparsity structure are expensive (consider LIL or DOK)

### BSR - Block Sparse Row

The Block Compressed Row (`BSR`) format is very similar to the Compressed Sparse Row (`CSR`) format. 

BSR is appropriate for sparse matrices with *dense sub matrices* like the example below. 

Block matrices often arise in *vector-valued* finite element discretizations. 

In such cases, BSR is **considerably more efficient** than CSR and CSC for many sparse arithmetic operations.

In [ ]:
from scipy.sparse import bsr_matrix

indptr = np.array([0, 2, 3, 6])
indices = np.array([0, 2, 2, 0, 1, 2])
data = np.array([1, 2, 3, 4, 5, 6]).repeat(4).reshape(6, 2, 2)
bsr_matrix((data,indices,indptr), shape=(6, 6)).toarray()

## COO - Coordinate Sparse Matrix

**Advantages of the CSC format**

    * facilitates fast conversion among sparse formats
    * permits duplicate entries (see example)
    * very fast conversion to and from CSR/CSC formats

**Disadvantages of the CSC format**

    * does not directly support arithmetic operations and slicing
    
** Intended Usage**

    * COO is a fast format for constructing sparse matrices
    * Once a matrix has been constructed, convert to CSR or CSC format for fast arithmetic and matrix vector
    operations
    * By default when converting to CSR or CSC format, duplicate (i,j) entries will be summed together. 
    This facilitates efficient construction of finite element matrices and the like.


## DOK - Dictionary of Keys

Sparse matrices can be used in arithmetic operations: they support addition, subtraction, multiplication, division, and matrix power.

Allows for efficient O(1) access of individual elements. Duplicates are not allowed. Can be efficiently converted to a coo_matrix once constructed.

In [ ]:
from scipy.sparse import dok_matrix
S = dok_matrix((5, 5), dtype=np.float32)
for i in range(5):
    for j in range(i, 5):
        S[i,j] = i+j
        
S.toarray()

The ``scipy.sparse`` submodule also has a lot of functions for sparse matrices
including linear algebra, sparse solvers, graph algorithms, and much more.

# Exercises

## Ex 1.1

Create a big numpy **dense** matrix filled with random numbers in 
`[0, 1)`.
Generate a random number within this range and subsitute all the elements in the matrix **less than** this number with a zero.

Save resulting matrix as a `DOK` sparse matrix

## Ex 1.2

Repeat the previous exercise, but this time use a `CSR` sparse matrix.

## Ex 1.3

Transform the previously generated sparse matrix back to a full dense `numpy.array`.

## Ex 1.4 

Generate two sparse Matrix and sum them together, choosing the most appropriate internal representation (i.e. `LIL`, `CSR`, `DOK`...).

#### Hint: Oh c'mon.. :)